# Remove Duplicates From the List
This algo is used to remove duplicate retailers from the list

<img src="remove_duplicates.jpg" width="400px" hieght="800px">

In [2]:
#########
# Import Libraries
#########
import sys  
import csv
import pymysql
import re
import pandas as pd
import numpy as np 
import pandasql as ps
import numpy as np
from sqlalchemy import create_engine
import pymysql
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

In [3]:
## Read StorePArtList with 6 digit pincode ###
my_df = pd.read_csv("DATA/PR retailers for internal duplication.csv",dtype=str,low_memory=False)
query1 = """
        select distinct Pincode from my_df where length(Pincode)=6
     """
df_pincode = ps.sqldf(query1, locals())
query1 = """
        SELECT Id as Id,lower(RetailerName) as StorePartyName,(lower(coalesce(Address1,'')) || ' ' || lower(coalesce(Address2,'')))as Address,lower(City) as City,lower(Pincode) as Pincode FROM my_df
        WHERE Pincode in (select Pincode from df_pincode)
     """
my_df = ps.sqldf(query1, locals())

final_merged_list = pd.DataFrame(columns=['Id','StorePartyName','Address','City','Pincode','StorePartyName_removed_unwanted_words','RetailerId','AddressPercentage'])
for idx1, r1 in df_pincode.iterrows():
    print(r1['Pincode'])
    query2 = """
            SELECT * FROM my_df WHERE lower(Pincode) = '{}'
         """.format(r1['Pincode'])
    df_retailers = ps.sqldf(query2, locals())

    #Remove Un-wanted Words from RetailerName 
    df_retailers['RetailerName_removed_unwanted_words'] = df_retailers['StorePartyName'].str.replace('medical and general stores|&|medigen|medgen|gen stores|gen store|genstores|med & gen|medical  gen|medi  gen|pharmacuticals|pharmacutical|distributors|distributor|agencies|agency|medicals|medical|meidcal|madical|medicos|medicose|medisales|medicare|stores|stotes|stors|stors|store|hospitals|hospital|surgicals|surgical|pharmaceuticals|pharmaceutical|pharmacies|pharmacy|pharma|chemists|chemist|druggists|medicines|medicine|medicos|medico|generals|general|genral|ganeral|generel|enterprises|enterprise|drugs |drug |drugist|druggist|centre|center|corner|shopee|wholsale|retail|cosmetics|healthcare|house|points|point|pvt|ltd| medi | med | gen | med | and', '')
    df_retailers

    try:
        Exact_Percentage = 0
        lst_df = pd.DataFrame(columns=['Id','StorePartyName','Address','City','Pincode','StorePartyName_removed_unwanted_words','RetailerId','AddressPercentage'])
        tmp_df = pd.DataFrame(columns=['AddressPercentage','RetailerId','Id'])
        c=1
        for index1, row1 in df_retailers.iterrows():
            print(c)
            c= c + 1
            Id = str(row1['Id'])
            StorePartyName = (re.sub('[^a-zA-Z ]','',str(row1['StorePartyName'])).strip())
            StorePartyName_removed_unwanted_words = (re.sub('[^a-zA-Z ]','',str(row1['RetailerName_removed_unwanted_words'])).strip())
            StorePartyAddress = str(row1['Address']).strip()        
            StorePartyPincode = re.sub('[^0-9 ]','',str(row1['Pincode'])).strip()
            StorePartyCity = (re.sub('[^a-zA-Z ]','',str(row1['City'])).strip())
            if(len(lst_df)>0):
                if len(StorePartyName_removed_unwanted_words)>2:
                    a = lst_df[(lst_df['StorePartyName']!="") & (lst_df['StorePartyName_removed_unwanted_words'].str.startswith(StorePartyName_removed_unwanted_words[:2]))]
                    if(len(a)>0):
                        df_retailer_mapping = (a[((a['StorePartyName_removed_unwanted_words'].str.contains(StorePartyName_removed_unwanted_words)) | (a.apply(lambda row: fuzz.token_set_ratio(row['StorePartyName_removed_unwanted_words'], StorePartyName_removed_unwanted_words), axis=1) > 92))])
                    else:
                        df_retailer_mapping = pd.DataFrame()
                else:
                    a = lst_df[(lst_df['StorePartyName']!="") & (lst_df['StorePartyName'].str.startswith(StorePartyName[:2]))]
                    if(len(a)>0):
                        df_retailer_mapping = (a[((a['StorePartyName'].str.contains(StorePartyName)) | (a.apply(lambda row: fuzz.token_set_ratio(row['StorePartyName'], StorePartyName), axis=1) > 92))])
                    else:
                        df_retailer_mapping = pd.DataFrame()
            else:
                df_retailer_mapping = pd.DataFrame()

            #print(df_retailer_mapping)

            if(len(df_retailer_mapping)>0):
                for index, row in df_retailer_mapping.iterrows():
                    Address_Ratio = fuzz.token_set_ratio(re.sub(' +', ' ', row['Address']),re.sub(' +', ' ', StorePartyAddress))
                    #print("Address Ration:",Address_Ratio)
                    if Address_Ratio >= 50:
                        ######Create Dataframe        #########
                        tmp_df.loc[index,'Id'] = Id
                        tmp_df.loc[index,'RetailerId'] = str(row['Id'])
                        tmp_df.loc[index,'AddressPercentage'] = Address_Ratio


                if (len(tmp_df)>0):            
                    query = "SELECT * from tmp_df order by AddressPercentage desc limit 1"
                    tmp_final = ps.sqldf(query, locals())
                    for index_s, row_max_s in tmp_final.iterrows():
                        lst_df = lst_df.append({'Id':str(row1['Id']),'StorePartyName': str(row1['StorePartyName']),'Address':str(row1['Address']),'City':str(row1['City']),'Pincode':str(row1['Pincode']),'StorePartyName_removed_unwanted_words':str(StorePartyName_removed_unwanted_words),'RetailerId':str(row_max_s['RetailerId']),'AddressPercentage':str(row_max_s['AddressPercentage'])}, ignore_index=True)

                    tmp_df = tmp_df.iloc[0:0]
                    tmp_final = tmp_final.iloc[0:0]
                else:
                    #print("else:")
                    RetailerId = ""
                    lst_df = lst_df.append({'Id':str(row1['Id']),'StorePartyName': str(row1['StorePartyName']),'Address':str(row1['Address']),'City':str(row1['City']),'Pincode':str(row1['Pincode']),'StorePartyName_removed_unwanted_words':str(StorePartyName_removed_unwanted_words),'RetailerId':str(RetailerId),'AddressPercentage':fuzz.token_set_ratio(row['Address'],StorePartyAddress)}, ignore_index=True)

            else:
                #print("main else:")
                RetailerId = ""
                lst_df = lst_df.append({'Id':str(row1['Id']),'StorePartyName': str(row1['StorePartyName']),'Address':str(row1['Address']),'City':str(row1['City']),'Pincode':str(row1['Pincode']),'StorePartyName_removed_unwanted_words':str(StorePartyName_removed_unwanted_words),'RetailerId':str(RetailerId),'AddressPercentage':''}, ignore_index=True)


        #print(lst_df)
        #lst_df.to_csv('/home/juned/PythonWork/Mapping/DATA/Mobile Number Retailer List_DuplicateRemoved_final-'+r1['Pincode']+'.csv', header=True, index=False)
        final_merged_list = final_merged_list.append(lst_df)
        lst_df = lst_df.iloc[0:0]
        #final_merged_list.to_csv('/home/juned/PythonWork/Mapping/DATA/Mobile Number Retailer List_DuplicateRemoved_Ubuntu-2000.csv', header=True, index=False)
    except IndexError:
        #  Python 3
        print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))                

final_merged_list.to_csv('/home/juned/PythonWork/Mapping/DATA/PR retailers for internal duplication-6digit.csv', header=True, index=False,quoting=csv.QUOTE_ALL)

In [4]:
# ## Read StorePArtList non-6 digit pincode ###
my_df = pd.read_csv("DATA/PR retailers for internal duplication.csv",dtype=str,low_memory=False)
query1 = """
        select distinct Pincode from my_df where length(Pincode)=6
     """
df_pincode = ps.sqldf(query1, locals())

query1 = """
        SELECT Id as Id,lower(RetailerName) as StorePartyName,(lower(coalesce(Address1,'')) || ' ' || lower(coalesce(Address2,'')))as Address,lower(City) as City,lower(Pincode) as Pincode FROM my_df
        WHERE Pincode not in (select Pincode from df_pincode)
     """
df_retailers = ps.sqldf(query1, locals())

# Remove Un-wanted Words from RetailerName 
df_retailers['RetailerName_removed_unwanted_words'] = df_retailers['StorePartyName'].str.replace('medical and general stores|&|medigen|medgen|gen stores|gen store|genstores|med & gen|medical  gen|medi  gen|pharmacuticals|pharmacutical|distributors|distributor|agencies|agency|medicals|medical|meidcal|madical|medicos|medicose|medisales|medicare|stores|stotes|stors|stors|store|hospitals|hospital|surgicals|surgical|pharmaceuticals|pharmaceutical|pharmacies|pharmacy|pharma|chemists|chemist|druggists|medicines|medicine|medicos|medico|generals|general|genral|ganeral|generel|enterprises|enterprise|drugs |drug |drugist|druggist|centre|center|corner|shopee|wholsale|retail|cosmetics|healthcare|house|points|point|pvt|ltd| medi | med | gen | med | and', '')
df_retailers

my_df_6digit = pd.read_csv("DATA/PR retailers for internal duplication-6digit.csv",dtype=str,low_memory=False)
# my_df_6digit     
lst_df = pd.DataFrame(columns=['Id','StorePartyName','Address','City','Pincode','StorePartyName_removed_unwanted_words','RetailerId','AddressPercentage'])
lst_df = my_df_6digit
# lst_df #640569
try:
    Exact_Percentage = 0
    
    tmp_df = pd.DataFrame(columns=['AddressPercentage','RetailerId','Id'])
    c=1
    for index1, row1 in df_retailers.iterrows():
        print(c)
        c= c + 1
        Id = str(row1['Id'])
        StorePartyName = (re.sub('[^a-zA-Z ]','',str(row1['StorePartyName'])).strip())
        StorePartyName_removed_unwanted_words = (re.sub('[^a-zA-Z ]','',str(row1['RetailerName_removed_unwanted_words'])).strip())
        StorePartyAddress = str(row1['Address']).strip()        
        StorePartyPincode = re.sub('[^0-9 ]','',str(row1['Pincode'])).strip()
        StorePartyCity = (re.sub('[^a-zA-Z ]','',str(row1['City'])).strip())
        if(len(lst_df)>0):
            if len(StorePartyName_removed_unwanted_words)>2:
                a = lst_df[(lst_df['StorePartyName']!="") & (lst_df['Id']!=Id) & (lst_df['StorePartyName_removed_unwanted_words'].str.startswith(StorePartyName_removed_unwanted_words[:2]))]
                if(len(a)>0):
                    df_retailer_mapping = (a[((a['StorePartyName_removed_unwanted_words'].str.contains(StorePartyName_removed_unwanted_words)) | (a.apply(lambda row: fuzz.token_set_ratio(row['StorePartyName_removed_unwanted_words'], StorePartyName_removed_unwanted_words), axis=1) > 92))])
                else:
                    df_retailer_mapping = pd.DataFrame()
            else:
                if len(StorePartyName)<=1:
                    df_retailer_mapping = pd.DataFrame()
                else:
                    a = lst_df[(lst_df['StorePartyName']!="") & (lst_df['Id']!=Id) & (lst_df['StorePartyName'].str.startswith(StorePartyName[:2]))]
                    if(len(a)>0):
                        df_retailer_mapping = (a[((a['StorePartyName'].str.contains(StorePartyName)) | (a.apply(lambda row: fuzz.token_set_ratio(row['StorePartyName'], StorePartyName), axis=1) > 92))])
                    else:
                        df_retailer_mapping = pd.DataFrame()
        else:
            df_retailer_mapping = pd.DataFrame()

        #print(df_retailer_mapping)

        if(len(df_retailer_mapping)>0):
            for index, row in df_retailer_mapping.iterrows():
                if len(StorePartyAddress)>6:
                    Address_Ratio = fuzz.token_set_ratio(re.sub(' +', ' ', row['Address']),re.sub(' +', ' ', StorePartyAddress))
                    #print("Address Ration:",Address_Ratio)
                    if Address_Ratio >= 85:
                        ######Create Dataframe        #########
                        tmp_df.loc[index,'Id'] = Id
                        tmp_df.loc[index,'RetailerId'] = str(row['Id'])
                        tmp_df.loc[index,'AddressPercentage'] = Address_Ratio


            if (len(tmp_df)>0):            
                query = "SELECT * from tmp_df order by AddressPercentage desc limit 1"
                tmp_final = ps.sqldf(query, locals())
                for index_s, row_max_s in tmp_final.iterrows():
                    lst_df = lst_df.append({'Id':str(row1['Id']),'StorePartyName': str(row1['StorePartyName']),'Address':str(row1['Address']),'City':str(row1['City']),'Pincode':str(row1['Pincode']),'StorePartyName_removed_unwanted_words':str(StorePartyName_removed_unwanted_words),'RetailerId':str(row_max_s['RetailerId']),'AddressPercentage':str(row_max_s['AddressPercentage'])}, ignore_index=True)

                tmp_df = tmp_df.iloc[0:0]
                tmp_final = tmp_final.iloc[0:0]
            else:
                #print("else:")
                RetailerId = ""
                lst_df = lst_df.append({'Id':str(row1['Id']),'StorePartyName': str(row1['StorePartyName']),'Address':str(row1['Address']),'City':str(row1['City']),'Pincode':str(row1['Pincode']),'StorePartyName_removed_unwanted_words':str(StorePartyName_removed_unwanted_words),'RetailerId':str(RetailerId),'AddressPercentage':fuzz.token_set_ratio(row['Address'],StorePartyAddress)}, ignore_index=True)

        else:
            #print("main else:")
            RetailerId = ""
            lst_df = lst_df.append({'Id':str(row1['Id']),'StorePartyName': str(row1['StorePartyName']),'Address':str(row1['Address']),'City':str(row1['City']),'Pincode':str(row1['Pincode']),'StorePartyName_removed_unwanted_words':str(StorePartyName_removed_unwanted_words),'RetailerId':str(RetailerId),'AddressPercentage':''}, ignore_index=True)


    #print(lst_df)
    lst_df.to_csv('/home/juned/PythonWork/Mapping/DATA/PR retailers for internal duplication-all-final.csv', header=True, index=False,quoting=csv.QUOTE_ALL)
#     lst_df = lst_df.iloc[0:0]
except IndexError:
    #  Python 3
    print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))                

In [9]:
# used to merge multiple csv files into one
import os
import glob
import pandas as pd
os.chdir("DATA/today")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "/home/juned/PythonWork/Mapping/DATA/today/combined_csv.csv", index=False, encoding='utf-8-sig')
